In [1]:
import numpy as np
import math
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import os

In [16]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/testA.csv")

## reference
- https://tianchi.aliyun.com/forum/postDetail?spm=5176.12586969.1002.21.3b30685680jNhK&postId=164618

###  Drop columns with only one value (policyCode)

In [3]:
def drop_column(df, col_name):
    """
    function to drop selected columns in the dataframe
    @param df: dataframe
    @param col_name: columns to be dropped
    return: dataframe after dropping the columns
    """
    df = df.drop(col_name, axis=1)
    return df

In [17]:
train = drop_column(train, ["id", "policyCode"])

test = drop_column(test, ["id", "policyCode"])

### Fill missing values
- categorical
    - employmentLength: as it is a categorical, create another type "NA" to fill missing values
    - title, employmentTitle, postCode: as only three records have missing values, fill with mode
- numeric
    - dti: fill 1000
    - revolUtil, pubRecBankruptcies: median
    - n0-n14: median

In [18]:
def fill_value(df, col_list, fill=0):
    """
    a function to fill missing values
    @param df: dataframe
    @param col_list: list of column names
    @param fill: int (0) or str (mean, mode, median)
    return: preprocessed dataframe
    """
    if type(fill) == int or fill not in ["mean", "median", "mode"]:
        df[col_list] = df[col_list].fillna(fill)
    else:
        if fill == "mean":
            val = df[col_list].mean()
            val_map = val.to_dict()
        elif fill == "median":
            val = df[col_list].median()
            val_map = val.to_dict()
        else:
            val = df[col_list].mode()
            val_map = val.loc[0].to_dict()
        
        print(val_map)
        
        for col in col_list:
            df[col] = df[col].replace(np.nan, val_map[col])
        
    return df

In [8]:
def fill_missing_value(df):
    """
    fill missing values in the dataframe
    @param df: dataframe
    """
    # later will encode it into digits
    df = fill_value(df, ["employmentLength"], fill = "NA")
    
    # use the mode to fill categorical features
    df = fill_value(df, ["title", "employmentTitle", "postCode"], fill="mode") 
    
    # original max is 999 (infinite), now fill missing value with 1000, later will discretize it into bins
    df = fill_value(df, ["dti"], fill=1000) 
    
    # below are skewed features
    df = fill_value(df, ["revolUtil", "pubRecBankruptcies"], fill="median") 
    df = fill_value(df, ["n%d" % i for i in range(15)], fill="median") 
    
    return df

In [19]:
train = fill_missing_value(train) 

{'title': 0.0, 'employmentTitle': 54.0, 'postCode': 134.0}
{'revolUtil': 52.1, 'pubRecBankruptcies': 0.0}
{'n0': 0.0, 'n1': 3.0, 'n2': 5.0, 'n3': 5.0, 'n4': 4.0, 'n5': 7.0, 'n6': 7.0, 'n7': 7.0, 'n8': 13.0, 'n9': 5.0, 'n10': 11.0, 'n11': 0.0, 'n12': 0.0, 'n13': 0.0, 'n14': 2.0}


In [20]:
test = fill_missing_value(test)

{'title': 0.0, 'employmentTitle': 54.0, 'postCode': 134.0}
{'revolUtil': 52.3, 'pubRecBankruptcies': 0.0}
{'n0': 0.0, 'n1': 3.0, 'n2': 5.0, 'n3': 5.0, 'n4': 4.0, 'n5': 7.0, 'n6': 7.0, 'n7': 7.0, 'n8': 13.0, 'n9': 5.0, 'n10': 11.0, 'n11': 0.0, 'n12': 0.0, 'n13': 0.0, 'n14': 2.0}


In [11]:
print(train.isnull().sum())
print(test.isnull().sum())

loanAmnt              0
term                  0
interestRate          0
installment           0
grade                 0
subGrade              0
employmentTitle       0
employmentLength      0
homeOwnership         0
annualIncome          0
verificationStatus    0
issueDate             0
isDefault             0
purpose               0
postCode              0
regionCode            0
dti                   0
delinquency_2years    0
ficoRangeLow          0
ficoRangeHigh         0
openAcc               0
pubRec                0
pubRecBankruptcies    0
revolBal              0
revolUtil             0
totalAcc              0
initialListStatus     0
applicationType       0
earliesCreditLine     0
title                 0
n0                    0
n1                    0
n2                    0
n3                    0
n4                    0
n5                    0
n6                    0
n7                    0
n8                    0
n9                    0
n10                   0
n11             

### Encode categorical features
- grade, subGrade, employmentLength

In [14]:
grade_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6}

employmentLength_dict = {'NA':0, '< 1 year':1, '1 year':2, '2 years':3, '3 years': 4, '4 years': 5,
                         '5 years':6, '6 years':7,'7 years':8, '8 years':9, '9 years':10, '10+ years':11}

def get_sub_grade(grade, sub):
    """
    encode subGrade into digits
    @param grade: the digit of the grade
    @param sub: letter of the subgrade
    """
    return grade*10 + int(sub[1])

def encode(df):
    """
    encode the categorical features (grade, subGrade, employmentLength) into digits
    @param df: dataframe
    """
    df['grade'] = df['grade'].apply(lambda x: x if x not in grade_dict else grade_dict[x])
    
    df['subGrade'] = df.apply(lambda row: get_sub_grade(row['grade'],row['subGrade']), axis=1)
    
    df['employmentLength'] = df['employmentLength'].apply(lambda x: employmentLength_dict[x])
    
    return df

In [21]:
train = encode(train)
test = encode(test)

In [22]:
train.head()

,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,35000.0,5,19.52,917.97,4,42,320.0,3,2,110000.0,2,2014-07-01,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,Aug-2001,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,18000.0,5,18.49,461.90,3,32,219843.0,6,0,46000.0,2,2012-08-01,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,May-2002,1723.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0
2,12000.0,5,16.99,298.17,3,33,31698.0,9,0,74000.0,2,2015-10-01,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,May-2006,0.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,11000.0,3,7.26,340.96,0,4,46854.0,11,1,118000.0,1,2015-08-01,0,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,May-1999,4.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,3000.0,3,12.99,101.07,2,22,54.0,0,1,29000.0,2,2016-03-01,0,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,Aug-1977,11.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


## Deal with date features
- transform the earliestCreditLine and issueDate into months
- compute the difference in months between earliestCreditLine and issueDate
- keep issueDate and earliestCreditLine (months) as cate_features

In [23]:
def trans_issueDate(issueDate):
    """
    convert issueDatet to number of months
    @param issueDate: YYYY-MM-DD
    """
    year, month, day = issueDate.split('-')
    return int(year)*12 + int(month) - 1


def trans_earliesCreditLine(earliesCreditLine):
    """
    convert earliesCreditLine to number of months
    @param earliesCreditLine: m-YYYY
    """
    month_dict = {"Jan":1, "Feb":2, "Mar":3, "Apr":4, "May":5, "Jun":6, "Jul":7, "Aug":8, "Sep":9, "Oct":10, "Nov":11, "Dec":12}
    month, year = earliesCreditLine.split('-')
    month = month_dict[month]
    
    return int(year)*12 + month - 1


def compute_month_diff(df):
    """
    compute the number months from earliesCreditLine to issueDate of the loan
    @param df: dataframe
    """
    
    df['issueDate'] = df['issueDate'].apply(lambda x: trans_issueDate(x))
    df['earliesCreditLine'] = df['earliesCreditLine'].apply(lambda x: trans_earliesCreditLine(x))
    df['month_diff'] = df['issueDate'] - df['earliesCreditLine']
    
    return df


In [24]:
train = compute_month_diff(train)

test = compute_month_diff(test)

In [25]:
train.head()

,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,month_diff
0,35000.0,5,19.52,917.97,4,42,320.0,3,2,110000.0,2,24174,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,24019,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,155
1,18000.0,5,18.49,461.90,3,32,219843.0,6,0,46000.0,2,24151,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,24028,1723.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,123
2,12000.0,5,16.99,298.17,3,33,31698.0,9,0,74000.0,2,24189,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,24076,0.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,113
3,11000.0,3,7.26,340.96,0,4,46854.0,11,1,118000.0,1,24187,0,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,23992,4.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,195
4,3000.0,3,12.99,101.07,2,22,54.0,0,1,29000.0,2,24194,0,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,23731,11.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,463


### Construct new features

In [26]:
def feature_interaction(df):
    """
    construct new features by feature interaction
    add 0.1 in denominator to avoid dividing by 0
    @param df: dataframe
    """
    
    # 贷款（本金加利息） / 信贷周转余额
    df['installment_term_revolBal'] = df['installment'] * 12 * df['term'] / (df['revolBal'] + 0.1)
    
    # 循环额度利用率 / 信贷周转余额
    df['revolUtil_revolBal'] = df['revolUtil'] / (df['revolBal'] + 0.1)
    
    # 借款人信用档案中 未结信用额度 / 当前的信用额度总数
    df['openAcc_totalAcc'] = df['openAcc'] / df['totalAcc']
    
    # 有的债务收入比为负值，取绝对值
    df['dti'] = np.abs(df['dti'])
    
    # 贷款金额占借款人所有债务的比例
    df['loanAmnt_dti_annualIncome'] = df['loanAmnt'] / (np.abs(df['dti']) * df['annualIncome'] + 0.1)
    
    # 收入比贷款额度
    df['annualIncome_loanAmnt'] = df['annualIncome'] / (df['loanAmnt'] + 0.1)
    
    # 收入比分期付款金额
    df['annualIncome_installment'] = df['annualIncome'] / (df['installment'] + 0.1)
    
    # 信贷周转余额 / 贷款金额
    df['revolBal_loanAmnt'] = df['revolBal'] / (df['loanAmnt'] + 0.1)
    
    # 信贷周转余额 / 分期付款金额
    df['revolBal_installment'] = df['revolBal'] / (df['installment'] + 0.1)

    
    return df

In [27]:
train = feature_interaction(train)

test = feature_interaction(test)

In [28]:
train.head()

,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,month_diff,installment_term_revolBal,revolUtil_revolBal,openAcc_totalAcc,loanAmnt_dti_annualIncome,annualIncome_loanAmnt,annualIncome_installment,revolBal_loanAmnt,revolBal_installment
0,35000.0,5,19.52,917.97,4,42,320.0,3,2,110000.0,2,24174,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,24019,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,155,2.278020,0.002022,0.259259,0.018662,3.142848,119.816572,0.690798,26.335682
1,18000.0,5,18.49,461.90,3,32,219843.0,6,0,46000.0,2,24151,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,24028,1723.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,123,1.835838,0.002577,0.722222,0.014061,2.555541,99.567100,0.838662,32.675325
2,12000.0,5,16.99,298.17,3,33,31698.0,9,0,74000.0,2,24189,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,24076,0.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,113,3.884023,0.011246,0.407407,0.007122,6.166615,248.097361,0.383830,15.442384
3,11000.0,3,7.26,340.96,0,4,46854.0,11,1,118000.0,1,24187,0,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,23992,4.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,195,1.233860,0.005287,0.321429,0.005417,10.727175,345.980179,0.904355,29.167888
4,3000.0,3,12.99,101.07,2,22,54.0,0,1,29000.0,2,24194,0,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,23731,11.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,463,1.236708,0.010877,0.444444,0.003217,9.666344,286.646239,0.980634,29.079767


### Discretize numeric features into bins by quantile
- 10 bins: annualIncome, loanAmnt
- 30 bins: interestRate, installment, revolBal, revolUtil, dti_bin

In [71]:
def discretize_bin(df, col_list, n_bins=10):
    """
    discretize continuous features into discrete bins
    @param df: datamframe
    @param col_list: list of feature names
    @param n_bins: number of bins
    """
    
    labels = list(range(n_bins))
    
    for col in col_list:
        df[col + "_bin"] = pd.qcut(df[col], n_bins, labels=labels, duplicates='drop')
    
    return df

In [72]:
# discretize into 10 bins
discretize_col_list = ["annualIncome", "loanAmnt"]

train = discretize_bin(train, discretize_col_list, n_bins=10)
test = discretize_bin(test, discretize_col_list, n_bins=10)


# discretize into 30 bins
discretize_col_list = ['interestRate', 'dti','installment','revolBal','revolUtil']

train = discretize_bin(train, discretize_col_list, n_bins=30)
test = discretize_bin(test, discretize_col_list, n_bins=30)

In [73]:
train.head()

,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,month_diff,installment_term_revolBal,revolUtil_revolBal,openAcc_totalAcc,loanAmnt_dti_annualIncome,annualIncome_loanAmnt,annualIncome_installment,revolBal_loanAmnt,revolBal_installment,annualIncome_bin,loanAmnt_bin,interestRate_bin,dti_bin,installment_bin,revolBal_bin,revolUtil_bin
0,35000.0,5,19.52,917.97,4,42,320.0,3,2,110000.0,2,24174,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,24019,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,155,2.278020,0.002022,0.259259,0.018662,3.142848,119.816572,0.690798,26.335682,8,9,27,14,28,24,13
1,18000.0,5,18.49,461.90,3,32,219843.0,6,0,46000.0,2,24151,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,24028,1723.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,123,1.835838,0.002577,0.722222,0.014061,2.555541,99.567100,0.838662,32.675325,2,6,25,25,18,19,9
2,12000.0,5,16.99,298.17,3,33,31698.0,9,0,74000.0,2,24189,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,24076,0.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,113,3.884023,0.011246,0.407407,0.007122,6.166615,248.097361,0.383830,15.442384,5,4,23,21,10,5,14
3,11000.0,3,7.26,340.96,0,4,46854.0,11,1,118000.0,1,24187,0,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,23992,4.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,195,1.233860,0.005287,0.321429,0.005417,10.727175,345.980179,0.904355,29.167888,8,4,2,14,13,13,15
4,3000.0,3,12.99,101.07,2,22,54.0,0,1,29000.0,2,24194,0,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,23731,11.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,463,1.236708,0.010877,0.444444,0.003217,9.666344,286.646239,0.980634,29.079767,0,0,15,28,1,2,6


### Compute ratios again median

In [74]:
cate_features = [
    "term", "grade", "subGrade", "employmentLength", "purpose", 'homeOwnership',
    "verificationStatus", "initialListStatus", "applicationType",
    "regionCode", "postCode", "title", "employmentTitle",
    "issueDate", "earliesCreditLine",
    'annualIncome_bin', 'loanAmnt_bin', 'installment_bin', 'interestRate_bin', 'revolBal_bin', 'revolUtil_bin', 'dti_bin' 
]


In [75]:
def tolist(df, col_name):
    """
    @param df: dataframe
    @param col_name: column name in str
    return the list of values in the column of dataframe
    """
    return list(set(df[col_name]))

In [76]:
ratio_feat_lst = [
    'loanAmnt', 'installment', 'interestRate', 'annualIncome', 'dti', 'openAcc', 'revolBal', 'revolUtil', 'totalAcc'
]

In [77]:
# get list of categorical features in the train set, and use the list to construct features in the train and test set

issueDate_lst = tolist(train,'issueDate')
employmentLength_lst = tolist(train,'employmentLength')
purpose_lst = tolist(train,'purpose')
homeOwnership_lst = tolist(train,'homeOwnership')

In [79]:
print(len(issueDate_lst))
print(len(employmentLength_lst))
print(len(purpose_lst))
print(len(homeOwnership_lst))

139
12
14
6


In [80]:
def mask(df, ratio_feat_lst, issueDate_lst, employmentLength_lst, purpose_lst, homeOwnership_lst):  
    """
    @param df: dataframe, train or test
    @param ratio_feat_lst: a list of feature columns to compute ratios
    @param issueDate_lst: list of issueDate values
    @param employmentLength_lst: list of employmentLength values
    @param purpose_lst: list of purpose values
    @param homeOwnership_lst: list of homeOwnership values
    """
    for feat in ratio_feat_lst:
        issueDate_feat_median = {}
        issueDate_feat_quantile = {}
        
        # dt is in number of months
        for dt in issueDate_lst:
            
            # 最近6个月
            mask_feat = (df['issueDate'] >= dt-3)&(df['issueDate'] <= dt+3)
            feat_series = df.loc[mask_feat, feat]

            # 最近6个月，该特征的中位数
            issueDate_feat_median[dt] = feat_series.median()

            # 该特征，在近6个月该特征取值范围的百分位
            quantile = feat_series.rank() / len(feat_series)
            issueDate_feat_quantile[dt] = {}
            for val, q in zip(feat_series, quantile):
                issueDate_feat_quantile[dt][val] = q

        df[feat+'_issueDate_median'] = df['issueDate'].apply(lambda x: issueDate_feat_median[x])
        df[feat+'_issueDate_quantile'] = df.apply(
            lambda row: issueDate_feat_quantile[row['issueDate']][row[feat]], axis=1
        )
    
    
        # 该特征，在不同employmentLength分组的中位数
        employmentLength_median = {}
        for et in employmentLength_lst:
            mask = df['employmentLength'] == et 
            employmentLength_median[et] = df.loc[mask, feat].median()
        
        # 该特征，在不同purpose分组的中位数
        purpose_median = {}
        for pp in purpose_lst:
            mask = df['purpose'] == pp
            purpose_median[pp] = df.loc[mask, feat].median()
        
        # 该特征，在不同houseOwnership分组的中位数
        homeOwnership_median = {}
        for ho in homeOwnership_lst:
            mask = df['homeOwnership'] == ho
            homeOwnership_median[ho] = df.loc[mask, feat].median()

        # 计算当前特征，和各中位数的比值
        df[feat+'_employmentLength_ratio'] = df.apply(
            lambda row: row[feat] / employmentLength_median[row['employmentLength']], axis=1
        )

        df[feat+'_purpose_ratio'] = df.apply(
            lambda row: row[feat] / purpose_median[row['purpose']], axis=1
        )

        df[feat+'_homeOwnership_ratio'] = df.apply(
            lambda row: row[feat] / homeOwnership_median[row['homeOwnership']], axis=1
        ) 
    
    return df

In [83]:
train = mask(train, ratio_feat_lst, issueDate_lst, employmentLength_lst, purpose_lst, homeOwnership_lst)

In [84]:
train.shape

(800000, 106)

In [85]:
train.head()

,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,month_diff,installment_term_revolBal,revolUtil_revolBal,openAcc_totalAcc,loanAmnt_dti_annualIncome,annualIncome_loanAmnt,annualIncome_installment,revolBal_loanAmnt,revolBal_installment,annualIncome_bin,loanAmnt_bin,interestRate_bin,dti_bin,installment_bin,revolBal_bin,revolUtil_bin,loanAmnt_issueDate_median,loanAmnt_issueDate_quantile,loanAmnt_employmentLength_ratio,loanAmnt_purpose_ratio,loanAmnt_homeOwnership_ratio,installment_issueDate_median,installment_issueDate_quantile,installment_employmentLength_ratio,installment_purpose_ratio,installment_homeOwnership_ratio,interestRate_issueDate_median,interestRate_issueDate_quantile,interestRate_employmentLength_ratio,interestRate_purpose_ratio,interestRate_homeOwnership_ratio,annualIncome_issueDate_median,annualIncome_issueDate_quantile,annualIncome_employmentLength_ratio,annualIncome_purpose_ratio,annualIncome_homeOwnership_ratio,dti_issueDate_median,dti_issueDate_quantile,dti_employmentLength_ratio,dti_purpose_ratio,dti_homeOwnership_ratio,openAcc_issueDate_median,openAcc_issueDate_quantile,openAcc_employmentLength_ratio,openAcc_purpose_ratio,openAcc_homeOwnership_ratio,revolBal_issueDate_median,revolBal_issueDate_quantile,revolBal_employmentLength_ratio,revolBal_purpose_ratio,revolBal_homeOwnership_ratio,revolUtil_issueDate_median,revolUtil_issueDate_quantile,revolUtil_employmentLength_ratio,revolUtil_purpose_ratio,revolUtil_homeOwnership_ratio,totalAcc_issueDate_median,totalAcc_issueDate_quantile,totalAcc_employmentLength_ratio,totalAcc_purpose_ratio,totalAcc_homeOwnership_ratio
0,35000.0,5,19.52,917.97,4,42,320.0,3,2,110000.0,2,24174,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,24019,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,155,2.278020,0.002022,0.259259,0.018662,3.142848,119.816572,0.690798,26.335682,8,9,27,14,28,24,13,12500.0,0.979841,2.916667,2.500000,2.916667,379.01,0.949716,2.589843,2.143788,2.558017,13.35,0.907823,1.532182,1.250480,1.532182,64000.0,0.859044,1.833333,1.466667,1.833333,17.55,0.478191,0.994749,1.239099,0.936813,11.0,0.173046,0.700000,0.777778,0.700000,11267.0,0.819472,2.446176,2.981993,2.277720,55.4,0.399228,0.956947,1.118993,1.018750,24.0,0.588235,1.227273,1.285714,1.173913
1,18000.0,5,18.49,461.90,3,32,219843.0,6,0,46000.0,2,24151,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,24028,1723.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,123,1.835838,0.002577,0.722222,0.014061,2.555541,99.567100,0.838662,32.675325,2,6,25,25,18,19,9,11500.0,0.749617,1.500000,1.294964,1.200000,372.36,0.641629,1.253562,1.154058,1.101230,14.09,0.825496,1.448492,1.410374,1.480384,60000.0,0.311231,0.724409,0.707692,0.621563,17.07,0.898402,1.604035,1.520765,1.577664,10.0,0.726314,1.181818,1.181818,1.181818,11676.0,0.631476,1.443281,1.325839,1.128420,60.6,0.224571,0.762745,0.729831,0.732580,22.0,0.343153,0.818182,0.750000,0.692308
2,12000.0,5,16.99,298.17,3,33,31698.0,9,0,74000.0,2,24189,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,24076,0.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,113,3.884023,0.011246,0.407407,0.007122,6.166615,248.097361,0.383830,15.442384,5,4,23,21,10,5,14,12500.0,0.464971,0.937500,0.863309,0.800000,377.04,0.335536,0.778776,0.744978,0.710876,11.99,0.846740,1.333595,1.295957,1.360288,65000.0,0.585433,1.138462,1.138462,0.999905,18.38,0.670724,1.285714,1.244262,1.290816,11.0,0.505394,1.000000,1.000000,1.000000,11588.0,0.163524,0.398926,0.404532,0.344297,52.1,0.495294,0.979206,0.971857,0.975518,24.0,0.610117,1.173913,1.125000,1.038462
3,110

In [86]:
train.to_csv("data/process_train.csv", index=False)

In [87]:
test = mask(test, ratio_feat_lst, issueDate_lst, employmentLength_lst, purpose_lst, homeOwnership_lst)

test.to_csv("data/process_test.csv", index=False)